# Fish Speech

### Prepare Model

In [ ]:
# For Chinese users, you probably want to use mirror to accelerate downloading
# !set HF_ENDPOINT=https://hf-mirror.com
# !export HF_ENDPOINT=https://hf-mirror.com 

!huggingface-cli download fishaudio/fish-speech-1.5 --local-dir stores/checkpoints/fish-speech-1.5/

## WebUI Inference

> You can use --compile to fuse CUDA kernels for faster inference (10x).

In [ ]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1

!python ../../fish-speech/tools/run_webui.py \
    --llama-checkpoint-path stores/checkpoints/fish-speech-1.5 \
    --decoder-checkpoint-path stores/checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth \
    --compile

## CLI Inference

### 1. Encode reference audio: 

> You should get a `fake.npy` file.

In [ ]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1

src_audio = "datas/audios/小敏/小敏_01.mp3"
!python ../../fish-speech/fish_speech/models/vqgan/inference.py \
    -i {src_audio} \
    --checkpoint-path "stores/checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth" \
    --device "mps"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

env: PYTORCH_ENABLE_MPS_FALLBACK=1
2025-02-13 14:32:20.030 | INFO     | __main__:load_model:46 - Loaded model: <All keys matched successfully>
2025-02-13 14:32:20.031 | INFO     | __main__:main:75 - Processing in-place reconstruction of /Users/WangHao/Sites/学习/LargeData/音频语素/小敏/小敏_01.mp3
2025-02-13 14:32:20.039 | INFO     | __main__:main:86 - Loaded audio with 9.22 seconds
2025-02-13 14:32:20.409 | INFO     | __main__:main:94 - Generated indices of shape torch.Size([8, 199])
2025-02-13 14:32:20.950 | INFO     | __main__:main:113 - Generated audio of shape torch.Size([1, 1, 407552]), equivalent to 9.24 seconds from 199 features, features/second: 21.53
2025-02-13 14:32:20.954 | INFO     | __main__:main:120 - Saved audio to fake.wav


### 2. Generate semantic tokens from text:

> This command will create a codes_N file in the working directory, where N is an integer starting from 0.

> You may want to use `--compile` to fuse CUDA kernels for faster inference (~30 tokens/second -> ~300 tokens/second).

In [6]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1

!python ../../fish-speech/fish_speech/models/text2semantic/inference.py \
    --text "你好呀我的朋友" \
    --prompt-text "昨天我出门你猜怎么着手机居然没电了这大热天的我连个导航都用不了你说气人不气人" \
    --prompt-tokens "fake.npy" \
    --checkpoint-path "stores/checkpoints/fish-speech-1.5" \
    --device "mps" \
    --num-samples 2 \
    --compile

env: PYTORCH_ENABLE_MPS_FALLBACK=1
2025-02-13 14:35:11.197 | INFO     | __main__:main:1056 - Loading model ...
2025-02-13 14:35:20.101 | INFO     | __main__:load_model:681 - Restored model from checkpoint
2025-02-13 14:35:20.102 | INFO     | __main__:load_model:687 - Using DualARTransformer
2025-02-13 14:35:20.102 | INFO     | __main__:load_model:695 - Compiling function...
2025-02-13 14:35:20.339 | INFO     | __main__:main:1070 - Time to load model: 9.14 seconds
2025-02-13 14:35:20.393 | INFO     | __main__:generate_long:788 - Encoded text: 你好呀我的朋友
2025-02-13 14:35:20.394 | INFO     | __main__:generate_long:806 - Generating sentence 1/1 of sample 1/2
/Users/WangHao/Sites/学习/Python/FunnyProject/fish-speech/fish_speech/models/text2semantic/llama.py:265: UserWarning: The operator 'aten::isin.Tensor_Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pyto

### 3. Generate speech from semantic tokens:

In [8]:
%env PYTORCH_ENABLE_MPS_FALLBACK=1

!python ../../fish-speech/fish_speech/models/vqgan/inference.py \
    -i "temp/codes_1.npy" \
    --checkpoint-path "stores/checkpoints/fish-speech-1.5/firefly-gan-vq-fsq-8x1024-21hz-generator.pth" \
    --device "mps"

from IPython.display import Audio, display
audio = Audio(filename="fake.wav")
display(audio)

env: PYTORCH_ENABLE_MPS_FALLBACK=1
2025-02-13 14:36:17.220 | INFO     | __main__:load_model:46 - Loaded model: <All keys matched successfully>
2025-02-13 14:36:17.221 | INFO     | __main__:main:99 - Processing precomputed indices from temp/codes_1.npy
2025-02-13 14:36:17.765 | INFO     | __main__:main:113 - Generated audio of shape torch.Size([1, 1, 75776]), equivalent to 1.72 seconds from 37 features, features/second: 21.53
2025-02-13 14:36:17.767 | INFO     | __main__:main:120 - Saved audio to fake.wav
